# 🧮 Laboratorio Programación No Lineal

[![Open in Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/boiro9/OptimizacionIA/main?filepath=Laboratorio_ProgramacionNoLineal.ipynb)

En esta práctica vamos a resolver en **Pyomo** los distintos problemas de **programación no lineal** que se han ilustrado en las diapositivas de las clases expositivas.  

Uno de los puntos importantes que trabajaremos será ver cómo se pueden obtener los **multiplicadores de lagrange** de un problema resuelto con Pyomo. Esta será muy relevante para verificar si la solución obtenidas cumple las condiciones de KKT.


Recordemos que formulación general de un problema de programación no lineal viene dada por:
$$
\begin{array}{rlll}
\min & \displaystyle f(\pmb{x}) & & \\
&&& \\
\text{s.a.} & g_{i}(\pmb{x})\leq 0 & &i=1,\ldots,m \\
	        & h_{j}(\pmb{x})=0     & &j=1,\ldots,l \\
\end{array}
$$
donde las funciones $f$, $g_{i}$ y $h_{j}$ pueden ser funciones no lineales y no convexas. 

# 🚀  Instalación de paquetes:

Los requerimientos para poder ejecutar este jupyter son los siguientes:

In [29]:
# Instalar pyomo y amplpy
!pip install pyomo amplpy -q
# Instalar los solvers: HiGHS, CBC, Couenne, Bonmin, Ipopt, SCIP y GCG
!python -m amplpy.modules install coin highs scip gcg -q

<div style="border:1px solid #f0ad4e; padding:10px; border-radius:5px; background-color:#fcf8e3;">
<strong>⚠️ Advertencia: </strong>  <br>
Si ya has instalado previamente estos paquetes, y utilizas el mismo entorno de Python, <b>NO</b> es necesario que los vuelvas a instalar.
</div>

## Ejemplo 1 (diapositiva página 10)

$$
\begin{array}{rl}
\min       & (x_{1}-\dfrac{3}{2})^{2}+(x_{2}-5)^{2} \\
\text{s.a.}& -x_{1}+x_{2} \leq 2 \\
           & x_{1}+2x_{2} \leq 7\\
           & 2x_{1}+2x_{2} \leq 11\\
		   & x_{1}\geq 0 \\
           & x_{2}\geq 0
\end{array}
$$

In [30]:
import pyomo.environ as pe

modelo1 = pe.ConcreteModel('Ejemplo 1')

# Variables
modelo1.x1 = pe.Var(within=pe.NonNegativeReals) 
modelo1.x2 = pe.Var(within=pe.NonNegativeReals) 

# Función objetivo
modelo1.obj = pe.Objective(expr=(modelo1.x1-3/2)**2+(modelo1.x2-5)**2,sense=pe.minimize)

# Restricción 1:
modelo1.cons1 = pe.Constraint(expr=-modelo1.x1 + modelo1.x2 <= 2)

# Restricción 2:
modelo1.cons2 = pe.Constraint(expr= modelo1.x1 + 2*modelo1.x2 <= 7)

# Restricción 3:
modelo1.cons3 = pe.Constraint(expr= 2*modelo1.x1 + 2*modelo1.x2 <= 11)


El solver que utilizaremos para resolver los problemas de programación no lineal será **Ipopt**. Lo definiremos a través del acceso a los solvers que nos proporciona `amplpy`. Es importante destacar que una vez lo tenemos definido, la siguiente vez que queramos usarlo no sería neceseario definirlo de nuevo.

In [31]:
from amplpy import modules
solver_name = "ipopt"  # Opciones: "highs", "cbc", "couenne", "bonmin", "ipopt", "scip", "gcg".
solver_ipopt = pe.SolverFactory(
    solver_name+"nl",
    executable=modules.find(solver_name),
    solve_io="nl"
)

In [32]:
results = solver_ipopt.solve(modelo1)
print('*** Solucion *** :')
print('x1:', modelo1.x1())
print('x2:', modelo1.x2())
print('Funcion objetivo: ',modelo1.obj())

*** Solucion *** :
x1: 1.0000000120126342
x2: 3.000000028241969
Funcion objetivo:  4.24999987501949


Si queremos inspeccionar el modelo para ver si lo hemos formulado correctamente podemos usar el método `pprint()`, que nos saca información muy detallada del mismo.

In [33]:
modelo1.pprint()

2 Var Declarations
    x1 : Size=1, Index=None
        Key  : Lower : Value              : Upper : Fixed : Stale : Domain
        None :     0 : 1.0000000120126342 :  None : False : False : NonNegativeReals
    x2 : Size=1, Index=None
        Key  : Lower : Value             : Upper : Fixed : Stale : Domain
        None :     0 : 3.000000028241969 :  None : False : False : NonNegativeReals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : (x1 - 1.5)**2 + (x2 - 5)**2

3 Constraint Declarations
    cons1 : Size=1, Index=None, Active=True
        Key  : Lower : Body      : Upper : Active
        None :  -Inf : - x1 + x2 :   2.0 :   True
    cons2 : Size=1, Index=None, Active=True
        Key  : Lower : Body      : Upper : Active
        None :  -Inf : x1 + 2*x2 :   7.0 :   True
    cons3 : Size=1, Index=None, Active=True
        Key  : Lower : Body        : Upper : Active
        None :  -In

## Ejemplo 6.1 (diapositiva página 18)

$$
\begin{array}{rl}
\min       & (x_{1}-3)^{2}+(x_{2}-2)^{2} \\
\text{s.a.}& x_{1}^{2}+x_{2}^{2} \leq 5 \\
           & x_{1}+x_{2} \leq 3\\
		   & x_{1}\geq 0 \\
           & x_{2}\geq 0
\end{array}
$$

In [34]:
import pyomo.environ as pe

modelo2 = pe.ConcreteModel("Ejemplo 6.1")

# Variables
modelo2.x1 = pe.Var(within=pe.NonNegativeReals) 
modelo2.x2 = pe.Var(within=pe.NonNegativeReals) 

# Función objetivo
modelo2.obj = pe.Objective(expr=(modelo2.x1-3)**2+(modelo2.x2-2)**2,sense=pe.minimize)

# Restricción 1:
modelo2.cons1 = pe.Constraint(expr=modelo2.x1**2 + modelo2.x2**2 <= 5)

# Restricción 2:
modelo2.cons2 = pe.Constraint(expr= modelo2.x1 + modelo2.x2 <= 3)


In [35]:
results = solver_ipopt.solve(modelo2)
print('*** Solucion *** :')
print('x1:', modelo2.x1())
print('x2:', modelo2.x2())
print('Funcion objetivo: ',modelo2.obj())

*** Solucion *** :
x1: 1.9999668104799597
x2: 1.0000332190655201
Funcion objetivo:  1.9999999431140907


In [36]:
# modelo2.pprint()

Para mirar qué restricciones están saturadas en la solución que ha encontrado el solver, podemos hacer un display del modelo: `display`. Simplemente nos tenemos que fijar si el "body" de la restricción (valor de la restricción evaluada en el punto), coincide con su cota inferior ("Lower") o superior ("Upper").

In [37]:
modelo2.display()

Model 'Ejemplo 6.1'

  Variables:
    x1 : Size=1, Index=None
        Key  : Lower : Value              : Upper : Fixed : Stale : Domain
        None :     0 : 1.9999668104799597 :  None : False : False : NonNegativeReals
    x2 : Size=1, Index=None
        Key  : Lower : Value              : Upper : Fixed : Stale : Domain
        None :     0 : 1.0000332190655201 :  None : False : False : NonNegativeReals

  Objectives:
    obj : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 1.9999999431140907

  Constraints:
    cons1 : Size=1
        Key  : Lower : Body               : Upper
        None :  None : 4.9999336822559295 :   5.0
    cons2 : Size=1
        Key  : Lower : Body             : Upper
        None :  None : 3.00000002954548 :   3.0


En este caso podemos ver que la **restricción 2** ($x_{1}+x_{2} \leq 3$) (`cons2`) está saturada porque su Body es de 3 ($x_{1}+x_{2}=2+1=3$) que coincide con el Upper.

## Ejemplo 6.2 (diapositiva página 22)

$$
\begin{array}{rl}
\min       & (x_{1}-3)^{2}+(x_{2}-2)^{2} \\
\text{s.a.}& x_{1}^{2}+x_{2}^{2} \leq 5 \\
           & x_{1}+2x_{2} \leq 4\\
		   & x_{1}\geq 0 \\
           & x_{2}\geq 0
\end{array}
$$

In [38]:
import pyomo.environ as pe

modelo3 = pe.ConcreteModel('Ejemplo 6.2')

# Variables
modelo3.x1 = pe.Var(within=pe.NonNegativeReals) 
modelo3.x2 = pe.Var(within=pe.NonNegativeReals) 

# Función objetivo
modelo3.obj = pe.Objective(expr=(modelo3.x1-3)**2+(modelo3.x2-2)**2,sense=pe.minimize)

# Restricción 1:
modelo3.cons1 = pe.Constraint(expr=modelo3.x1**2 + modelo3.x2**2 <= 5)

# Restricción 2:
modelo3.cons2 = pe.Constraint(expr= modelo3.x1 + 2*modelo3.x2 <= 4)


In [39]:
results = solver_ipopt.solve(modelo3)
print('*** Solucion *** :')
print('x1:', modelo3.x1())
print('x2:', modelo3.x2())
print('Funcion objetivo: ',modelo3.obj())

*** Solucion *** :
x1: 2.0000000020797604
x2: 1.000000017077632
Funcion objetivo:  1.9999999616852158


In [40]:
#modelo3.pprint()

Veamos qué restricciones están saturadas usando el método `display()`.

In [41]:
modelo3.display()

Model 'Ejemplo 6.2'

  Variables:
    x1 : Size=1, Index=None
        Key  : Lower : Value              : Upper : Fixed : Stale : Domain
        None :     0 : 2.0000000020797604 :  None : False : False : NonNegativeReals
    x2 : Size=1, Index=None
        Key  : Lower : Value             : Upper : Fixed : Stale : Domain
        None :     0 : 1.000000017077632 :  None : False : False : NonNegativeReals

  Objectives:
    obj : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 1.9999999616852158

  Constraints:
    cons1 : Size=1
        Key  : Lower : Body              : Upper
        None :  None : 5.000000042474306 :   5.0
    cons2 : Size=1
        Key  : Lower : Body              : Upper
        None :  None : 4.000000036235024 :   4.0


En este caso vemos que las dos restricciones, `cons1` y `cons2`, están saturadas

<div style="border: 2px solid #4CAF50; background-color: #E8F5E9; padding: 10px; border-radius: 8px;">
<b>💡 Obtención de multiplicadores de lagrange con Pyomo:</b>  
<br>Cuando se formula un problema en Pyomo, es posible pedirle que nos devuelva información acerca de las variables duales (recuerda que existe una equivalencia entre variables duales y multiplicadores de Lagrange bajo ciertas condiciones (ver <b>Teorema 6.8</b>)). Para que Pyomo nos devuelva esta información debemos, antes de llamar al solver, crear un nuevo atributo (<code>.dual</code>) al objeto con el modelo (<code>modelo</code>) que le indique al solver que envíe dicha información, de la siguiente manera:<br>
    
``modelo1.dual = pe.Suffix(direction=pe.Suffix.IMPORT)``
</div>


In [42]:
modelo3.dual = pe.Suffix(direction=pe.Suffix.IMPORT)
results = solver_ipopt.solve(modelo3)
#print("Duals")
#for c in modelo1.component_objects(pe.Constraint, active=True):
#    print("   Constraint", c)
#    for index in c:
#        print("      ", index, modelo1.dual[c[index]])

In [43]:
modelo3.dual.pprint()

dual : Direction=IMPORT, Datatype=FLOAT
    Key   : Value
    cons1 : -0.3333333390164021
    cons2 : -0.6666666395379771


In [44]:
modelo3.dual[modelo3.cons1]

-0.3333333390164021

In [45]:
modelo3.dual[modelo3.cons2]

-0.6666666395379771

<div style="border:2px solid #f0ad4e; padding:10px; border-radius:5px; background-color:#fcf8e3;">
<b>⚠️ IMPORTANTE: </b>  <br>
Es importante tener en cuenta que Pyomo devuelve los multiplicadores con <b>signo contrario</b> al que obtuvimos en clase:
$$
u_1 \approx \frac{1}{3} \qquad \qquad \qquad u_2 \approx \frac{2}{3}
$$
</div>

## Ejemplo 6.2 (diapositiva página 22)

$$
\begin{array}{rl}
\min       & (x_{1}-3)^{2}+(x_{2}-2)^{2} \\
\text{s.a.}& x_{1}^{2}+x_{2}^{2} \leq b_{1} \\
           & x_{1}+2x_{2} \leq b_{2}\\
		   & x_{1}\geq 0 \\
           & x_{2}\geq 0
\end{array}
$$

<div style="border: 2px solid #4CAF50; background-color: #E8F5E9; padding: 12px; border-radius: 8px; line-height: 1.5;">
<b>💡 Uso de parámetros en Pyomo:</b><br>
Como en este ejemplo queremos resolver varias veces el mismo problema simplemente cambiando los lados derechos de las restricciones 
$b_1$ y $b_2$, emplearemos el objeto <code>Param</code> de Pyomo. De esta manera, si los definimos como <code>Param</code> podemos 
especificarle un valor inicial (<code>initialize</code>) y también debemos definir <code>mutable=True</code> para que nos permita cambiar 
el valor de los mismos y resolverlo de nuevo sin necesidad de recrear el modelo desde cero.

<code>
modelo4.b1 = pe.Param(initialize=5.1, mutable=True)
modelo4.b2 = pe.Param(initialize=4, mutable=True)
</code>
</div>


In [46]:
import pyomo.environ as pe

modelo4 = pe.ConcreteModel('Ejemplo 6.2 (sensibilidad)')

modelo4.b1 = pe.Param(initialize=5.1, mutable=True)
modelo4.b2 = pe.Param(initialize=4, mutable=True)

# Variables
modelo4.x1 = pe.Var(within=pe.NonNegativeReals) 
modelo4.x2 = pe.Var(within=pe.NonNegativeReals) 

# Función objetivo
modelo4.obj = pe.Objective(expr=(modelo4.x1-3)**2+(modelo4.x2-2)**2,sense=pe.minimize)

# Restricción 1:
modelo4.cons1 = pe.Constraint(expr=modelo4.x1**2 + modelo4.x2**2 <= modelo4.b1)

# Restricción 2:
modelo4.cons2 = pe.Constraint(expr= modelo4.x1 + 2*modelo4.x2 <=  modelo4.b2)

**Variación 1**: Resolvemos para $b_1=5.1$ y $b_2=4$: 

In [47]:
results = solver_ipopt.solve(modelo4)
print('*** Solucion *** :')
print('x1:', modelo4.x1())
print('x2:', modelo4.x2())
print('Funcion objetivo: ',modelo4.obj())

*** Solucion *** :
x1: 2.0328828027391457
x2: 0.9835586168988126
Funcion objetivo:  1.968468758518345


**Variación 2**: Resolvemos para $b_1=5$ y $b_2=4.1$: 

In [48]:
modelo4.b1 = 5
modelo4.b2 = 4.1
results = solver_ipopt.solve(modelo4)
print('*** Solucion *** :')
print('x1:', modelo4.x1())
print('x2:', modelo4.x2())
print('Funcion objetivo: ',modelo4.obj())

*** Solucion *** :
x1: 1.9647270443631604
x2: 1.0676364949302408
Funcion objetivo:  1.9410917982590046


**Variación 3**: Resolvemos para $b_1=6$ y $b_2=4$: 

In [49]:
modelo4.b1 = 6
modelo4.b2 = 4
results = solver_ipopt.solve(modelo4)
print('*** Solucion *** :')
print('x1:', modelo4.x1())
print('x2:', modelo4.x2())
print('Funcion objetivo: ',modelo4.obj())

*** Solucion *** :
x1: 2.2966629519353865
x2: 0.8516685428870004
Funcion objetivo:  1.8133481385755095


**Variación 4**: Resolvemos para $b_1=5$ y $b_2=5$: 

In [50]:
modelo4.b1 = 5
modelo4.b2 = 5
results = solver_ipopt.solve(modelo4)
print('*** Solucion *** :')
print('x1:', modelo4.x1())
print('x2:', modelo4.x2())
print('Funcion objetivo: ',modelo4.obj())

*** Solucion *** :
x1: 1.8605210279471032
x2: 1.2403473507360883
Funcion objetivo:  1.8754844752844062
